# Caltrans Data Extraction

The goal of this project is to extract relevant data from text files, previously converted from PDF files. Since the text files are quite structured, the decision is to use regex to do this.

## Setup

Install the following packages if you don't have them yet:

In [1]:
# pip install pandas numpy tqdm ipykernel notebook python-dotenv openpyxl

In [2]:
from pathlib import Path
from pprint import pprint
from typing import List, Tuple
from collections import defaultdict
import re
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import os
from datetime import datetime
import shutil
from dotenv import load_dotenv

from utils import *

%reload_ext autoreload
%autoreload 2

# pd.set_option('display.max_rows', None)  # optional to see all rows in DataFrames

## Test parse_subcontracted_line_item

In [3]:
df = pd.read_csv('subcontracted_line_item_examples.txt', header=None, delimiter="\t", names=['Subcontracted_Line_Item'])
df, df_outlier = parse_subcontracted_line_item(df)
df

,Subcontracted_Line_Item,PARSED_1,PARSED_2,PARSED_3,PARSED_4,PARSED_5
0,ITEMS 6 THRU 8 AND 13 THRU 15,NaN,ITEMS,"6 - 8 , 13 - 15",NaN,"6, 7, 8, 13, 14, 15"
1,ITEM 12,NaN,ITEM,12,NaN,12
2,ITEMS 1 THRU 3 AND 5,NaN,ITEMS,"1 - 3 , 5",NaN,"1, 2, 3, 5"
3,ITEMS 6 THRU 8 AND 13 THRU 15 (PARTIALS),NaN,ITEMS,"6 - 8 , 13 - 15",(PARTIALS),"6, 7, 8, 13, 14, 15"
4,"ITEMS 6, 7, 8, 13, 14 AND 15",NaN,ITEMS,"6, 7, 8, 13, 14 , 15",NaN,"6, 7, 8, 13, 14, 15"
5,"ITEMS 4, 6 THRU 8, 13 THRU 15",NaN,ITEMS,"4, 6 - 8, 13 - 15",NaN,"4, 6, 7, 8, 13, 14, 15"
6,"ITEMS 6, 7, 8 AND 13 THRU 15",NaN,ITEMS,"6, 7, 8 , 13 - 15",NaN,"6, 7, 8, 13, 14, 15"
7,ITEM 94 (100%),NaN,ITEM,94,(100%),94
8,ITEM LESS GC PROVIDED WATER,NaN,ITEM,LESS GC PROVIDED WATER,NaN,COULD NOT PARSE
9,"WORK AS DESCRIBED BY BID ITEM NUMBERS 11,14,18",WORK AS DESCRIBED BY BID,ITEM NUMBERS,"11,14,18",NaN,"11, 14, 18"


In [4]:
df_outlier

,Subcontracted_Line_Item,PARSED_1,PARSED_2,PARSED_3,PARSED_4,PARSED_5
8,ITEM LESS GC PROVIDED WATER,NaN,ITEM,LESS GC PROVIDED WATER,NaN,COULD NOT PARSE
10,BID ITEMS 108-110 ALL 100%,BID,ITEMS,108-110 ALL 100%,NaN,COULD NOT PARSE
11,WORK AS DESCRIBED BY BID ITEMS LISTED,WORK AS DESCRIBED BY BID,ITEMS,LISTED,NaN,COULD NOT PARSE
12,"BID ITEMS 23-25,27-31,45,68-79 ALL 100%",BID,ITEMS,"23-25,27-31,45,68-79 ALL 100%",NaN,COULD NOT PARSE
14,"BID ITEMS 20-29, EACH 100%",BID,ITEMS,"20-29, EACH 100%",NaN,COULD NOT PARSE
15,"ITEMS 91,92,93,94,95,98 - 100%",NaN,ITEMS,"91,92,93,94,95,98 - 100%",NaN,COULD NOT PARSE
16,"ITEMS 10, 18 & -23",NaN,ITEMS,"10, 18 , -23",NaN,COULD NOT PARSE
17,"ITEMS 2, 3, 4 6 & 5",NaN,ITEMS,"2, 3, 4 6 , 5",NaN,COULD NOT PARSE


# One sample study

In [5]:
# filepath = RAW_DATA_PATH.parent / 'sample' / '01-0A3804.pdf_2724.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A3804.pdf_4353.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A0904.pdf_2724.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A1204.pdf_11468.txt'
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0F4304.pdf_12346.txt'  # issue # 11
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0K6104.pdf_12731.txt'  # issue # 9
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0K4604.pdf_12040.txt'  # issue # 1
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0H3204.pdf_9871.txt'  # issue # 5
# filepath = RAW_DATA_PATH_LINEPRINTER / '01-0A0404.pdf_10165.txt'  # different format
filepath = RAW_DATA_PATH_LINEPRINTER / '04-4G6404.pdf_7310.txt'  # different format


In [6]:
contract_number_from_filename, tag, identifier = get_contract_number_and_tag_from_filename(filepath.stem)
file_contents = read_file(filepath)

In [7]:
contract_number_from_filename, tag, identifier 

('04-4G6404', '7310', '04-4G6404_7310')

Extract contract data:

In [8]:
df_contract_data = pd.DataFrame([extract_contract_data(file_contents, identifier)])
df_contract_bid_data = pd.DataFrame(extract_contract_bid_data(file_contents, identifier))
df_bid_subcontractor_data = parse_subcontracted_line_item(fill_gaps_in_bidder_id(pd.DataFrame(extract_bid_subcontractor_data(file_contents, identifier))))
df_contract_line_item_data = pd.DataFrame(extract_contract_line_item_data(file_contents, identifier))

In [9]:
df_contract_data

,Identifier,Postponed_Contract,Bid_Opening_Date,Contract_Date,Contract_Code,Number_of_Contract_Items,Total_Number_of_Working_Days,Number_of_Bidders,Engineers_Est,Amount_Over,Amount_Under,Percent_Est_Over,Percent_Est_Under,Contract_Description
0,04-4G6404_7310,0,11/03/15,11/05/15,D,121,90,8,"4,178,805.00",,"791,668.00",,18.94,ROUTES 84/280 SEPARATION CONSTRUCT


In [10]:
df_contract_bid_data

,Identifier,Bid_Rank,A_plus_B_indicator,Bid_Total,Bidder_ID,Bidder_Name,Bidder_Phone,Extra,Weird_Contract_Notes,CSLB_Number
0,04-4G6404_7310,1,1,"3,657,137.00",3,GHILOTTI CONSTRUCTION CO. INC.,707 585-1221,,,00644515
1,04-4G6404_7310,2,1,"4,039,158.00",8,"GORDON N. BALL, INC.",925 838-5675,,,00710807
2,04-4G6404_7310,3,1,"4,189,774.00",4,"RGW CONSTRUCTION, INC.",925 606-2400,,,00591940
3,04-4G6404_7310,4,1,"4,298,995.26",1,GRANITE CONSTRUCTION,408 327-7013,,,00000089
4,04-4G6404_7310,5,1,"4,519,443.00",5,GRANITE ROCK COMPANY,408 574-1400,,,00000022
5,04-4G6404_7310,6,1,"4,609,762.00",2,"DISNEY CONSTRUCTION, INC.",650 259-9545,,,00866974
6,04-4G6404_7310,7,1,"4,768,691.00",6,VALENTINE CORPORATION,415 453-3732,,,00229225
7,04-4G6404_7310,8,1,"5,255,322.00",7,BUGLER CONSTRUCTION,925 416-0700,,,00740863


In [11]:
df_bid_subcontractor_data

(         Identifier Bidder_ID               Subcontractor_Name  \
 0    04-4G6404_7310        03  AVAR CONSTRUCTION SYSTEMS, INC.   
 1    04-4G6404_7310        03  AVAR CONSTRUCTION SYSTEMS, INC.   
 2    04-4G6404_7310        03  AVAR CONSTRUCTION SYSTEMS, INC.   
 3    04-4G6404_7310        03      CAMBLIN STEEL SERVICES INC.   
 4    04-4G6404_7310        03      CAMBLIN STEEL SERVICES INC.   
 ..              ...       ...                              ...   
 431  04-4G6404_7310        07           PACIFIC COAST DRILLING   
 432  04-4G6404_7310        07           PACIFIC COAST DRILLING   
 433  04-4G6404_7310        07           PACIFIC COAST DRILLING   
 434  04-4G6404_7310        07        R AN B PROTETIVE COATINGS   
 435  04-4G6404_7310        07        R AN B PROTETIVE COATINGS   
 
                   Subcontracted_Line_Item           City  \
 0                          ITEM 77 (100%)     FREMONT CA   
 1                          ITEM 95 (100%)     FREMONT OR   
 2         

In [15]:
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None, 
                       'display.width', None, 
                       'display.max_colwidth', None):
    display(df_contract_line_item_data)

,Identifier,Item_Number,Extra,Item_Code,Item_Description,Item_Dollar_Amount
0,04-4G6404_7310,1,,070030,LEAD COMPLIANCE PLAN,"1,500.00"
1,04-4G6404_7310,2,,120090,CONSTRUCTION AREA SIGNS,"5,000.00"
2,04-4G6404_7310,3,,120100,TRAFFIC CONTROL SYSTEM,"35,000.00"
3,04-4G6404_7310,4,,120159,TEMPORARY TRAFFIC STRIPE (PAINT),"1,206.00"
4,04-4G6404_7310,5,,120165,CHANNELIZER (SURFACE MOUNTED),540.00
5,04-4G6404_7310,6,,128652,PORTABLE CHANGEABLE MESSAGE SIGN (LS),"14,000.00"
6,04-4G6404_7310,7,,129000,TEMPORARY RAILING (TYPE K),"21,000.00"
7,04-4G6404_7310,8,,029897,TEMPORARY ALTERNATIVE CRASH CUSHION,"5,000.00"
8,04-4G6404_7310,9,,129150,TEMPORARY TRAFFIC SCREEN,"5,600.00"
9,04-4G6404_7310,10,,130100,JOB SITE MANAGEMENT,"10,000.00"


# Batch run

Here we run a batch or a single file (for example, if you want to run a specific outlier, make `files` a single element list):

In [25]:
files = RAW_DATA_PATH_LINEPRINTER.glob('*.txt')
# files = [RAW_DATA_PATH_LINEPRINTER/'01-0F9204.PDF_12364.txt']

In [26]:
contract_data = []
contract_bid_data = []
bid_subcontractor_data = []
contract_line_item_data = []
other_format = []
error_files = []

for filepath in tqdm(files):
    try:
        filename = filepath.stem
        file_contents = read_file(filepath)
        
        contract_number_from_filename, tag, identifier = get_contract_number_and_tag_from_filename(filename)
        contract_number_from_contents = get_contract_number(file_contents)
        
        if contract_number_from_filename == contract_number_from_contents:  
            contract_data.append(extract_contract_data(file_contents, identifier))
            contract_bid_data.extend(extract_contract_bid_data(file_contents, identifier))
            bid_subcontractor_data.extend(extract_bid_subcontractor_data(file_contents, identifier))
            contract_line_item_data.extend(extract_contract_line_item_data(file_contents, identifier))
        else:
            # if contract number doesn't match then something is off that needs investigation
            other_format.append({'other_format_filename': filename})
            # let's also copy the pdf to a folder for manual inspection
            shutil.copy(
                RAW_DATA_PATH_PDF / f'{filename}.pdf', 
                OUTLIERS_PATH_PDF / f'{filename}.pdf'
                )
            
            shutil.copy(
                RAW_DATA_PATH_LINEPRINTER / f'{filename}.txt', 
                OUTLIERS_PATH_LINEPRINTER / f'{filename}.txt'
                )
            
            shutil.copy(
                RAW_DATA_PATH_TABLE / f'{filename}.txt',
                OUTLIERS_PATH_TABLE / f'{filename}.txt'
                )     
    except Exception as e:
        error_files.append({'error_files': filename, 'error': e})
        
        

8977it [03:54, 38.23it/s]


# Save files

Uncomment the timestamp line if you want to save all files with the timstampt prefix:

In [27]:
# timestamp = datetime.strftime(datetime.now(), '%m-%d-%Y-%H:%M:%S')
timestamp = None

In [28]:
write_to_results(contract_data, "contract_data", timestamp=timestamp)
write_to_results(contract_bid_data, "contract_bid_data", timestamp=timestamp)

df_bid_subcontractor_data, df_bid_subcontractor_data_could_not_parse = parse_subcontracted_line_item(
    fill_gaps_in_bidder_id(pd.DataFrame(bid_subcontractor_data)))

write_to_results(df_bid_subcontractor_data, "bid_subcontractor_data", timestamp=timestamp)
write_to_results(df_bid_subcontractor_data_could_not_parse, "bid_subcontractor_outliers", timestamp=timestamp)

write_to_results(contract_line_item_data, "contract_line_item_data", timestamp=timestamp)
write_to_results(other_format, "other_format", timestamp=timestamp)

write_to_results(error_files, "errors", timestamp=timestamp)

# Save to excel

In [29]:
# Paths to your CSV files
csv_file_paths = RESULTS_PATH.glob('*.csv')

# Path to the output Excel file
excel_file_path = RESULTS_PATH / 'results.xlsx'

# Create a Pandas Excel writer using openpyxl as the engine
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    # Iterate over your CSV files
    for csv_file in tqdm(csv_file_paths):
        # Use Path from pathlib to work with file paths
        csv_path = Path(csv_file)
        
        # Extract the file name without the extension for the sheet name
        sheet_name = csv_path.stem
        
        # Read each CSV file into a DataFrame
        df = pd.read_csv(csv_file)
        
        # Write the DataFrame to a new sheet in the Excel file using the file name as the sheet name
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f'Merged CSV files into {excel_file_path}')

Merged CSV files into results/results.xlsx


In [31]:
import re

# Your original regex pattern
pattern = r'^\s+(\d+)\s+(\(F\))?\s+(\d+)\s+(.{45})\s+(.{35})\s+([\d,]+\.\d{2})'

# Sample input text
text = """
        11     130200      PREPARE WATER POLLUTION CONTROL PROGRAM        LS     LUMP SUM              800.00           800.00
        12     130530      TEMPORARY HYDRAULIC MULCH                      SQYD      3,650                1.00         3,650.00
                           (BONDED FIBER MATRIX)
        13     130570      TEMPORARY COVER                                SQYD      1,770                2.50         4,425.00
        14     130610      TEMPORARY CHECK DAM                            LF          170                6.00         1,020.00
        15     130620      TEMPORARY DRAINAGE INLET PROTECTION            EA            4              175.00           700.00
"""

# Process text to attach additional lines directly to the preceding line
lines = text.split('\n')
processed_lines = []
for i, line in enumerate(lines):
    if i > 0 and re.match(r'^\s{27}', line) and not re.match(pattern, line):
        processed_lines[-1] = processed_lines[-1].strip() + ' ' + line.strip()
    else:
        processed_lines.append(line)

# Now, apply the regex to each processed line
for line in processed_lines:
    match = re.match(pattern, line)
    if match:
        print(match.groups())


['',
 '        11     130200      PREPARE WATER POLLUTION CONTROL PROGRAM        '
 'LS     LUMP SUM              800.00           800.00',
 '        12     130530      TEMPORARY HYDRAULIC MULCH                      '
 'SQYD      3,650                1.00         3,650.00',
 '                           (BONDED FIBER MATRIX)',
 '        13     130570      TEMPORARY COVER                                '
 'SQYD      1,770                2.50         4,425.00',
 '        14     130610      TEMPORARY CHECK DAM                            '
 'LF          170                6.00         1,020.00',
 '        15     130620      TEMPORARY DRAINAGE INLET PROTECTION            '
 'EA            4              175.00           700.00',
 '']


('11', None, '130200', 'PREPARE WATER POLLUTION CONTROL PROGRAM      ', 'LS     LUMP SUM              800.00', '800.00')
('13', None, '130570', 'TEMPORARY COVER                              ', 'SQYD      1,770                2.50', '4,425.00')
('14', None, '130610', 'TEMPORARY CHECK DAM                          ', 'LF          170                6.00', '1,020.00')
('15', None, '130620', 'TEMPORARY DRAINAGE INLET PROTECTION          ', 'EA            4              175.00', '700.00')
